In [9]:
import os

import pdfplumber
from openai import OpenAI
from rich import print

client = OpenAI()

def query(question):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": question,
            }
        ],
        model="gpt-4o",
    )
    return chat_completion.choices[0].message.content


In [10]:
fname = os.path.expanduser("~/data/texas_air/RN103209227_112089_5-13-2016_1712046965 (2).pdf")
with pdfplumber.open(fname) as pdf:
    fulltext = "\n".join([page.extract_text() for page in pdf.pages])

print(fulltext[:1000] + "\n\n[...]\n\n" + fulltext[-500:])


EMISSION SOURCES - MAXIMUM ALLOWABLE EMISSION RATES
Permit Number 82739
This table lists the maximum allowable emission rates and all sources of air contaminants on the
applicants property covered by this permit. The emission rates shown are those derived from
information submitted as part of the application for permit and are the maximum rates allowed for
these facilities. Any proposed increase in emission rates may require an application for a
modification of the facilities covered by this permit.
AIR CONTAMINANTS DATA
Emission Source Air Contaminant Emission Rates
Point No. (1) Name (2) Name (3) lb/hr TPY(4)
LC-1 Lump Crusher (5) PM 0.02 0.03
PM 0.01 0.02
10
SC-2 Sifting Screen (5) PM 0.04 0.06
PM 0.01 0.02
10
LC-3 Conveyor Transfer (5) PM <0.01 <0.01
PM <0.01 <0.01
10
1 Cement Silo Baghouse PM 0.01 0.02
10
4, 5 Sand and Aggregate PM 0.39 0.58
Transfer To Hopper (5) PM 0.19 0.28
10
2, 6 Sand/Aggregate PM --- 0.02
Stockpile (5) PM 0.01
10
3, 7 Iron Ore Stockpile (5) PM --- 1.07
PM 0.

[...]

n the atmosphere, including PM and
10
PM
2.5
PM - particulate matter equal to or less than 10 microns in diameter
10
PM - particulate matter equal to or less than 2.5 microns in diameter
2.5
HAP - hazardous air pollutant as listed in § 112(b) of the Federal Clean Air
Act or Title 40 Code of Federal Regulations Part 63, Subpart C
(4) Compliance with annual emission limits is based on a rolling 12-month period.
(5) Fugitive emissions are an estimate only.
Dated June 25, 2010
Project Number: 132323

In [11]:
prompt = f"""You are a helpful data extraction AI reconstructing a table from text extracted from a PDF.
The columns to extract are "Emission Source", "Source Name", "Air Contaminant Name", "Emission Rate lbs/hr", "Emission Rate tons/year"

In the PDF, table cells that are left empty are implicitly repeated until a new value is found.
The reconstructed table shaould have all cells filled in. Please follow the below example to
reconstruct the full table from the text that follows the example.

Here is an example of extracted text:

EMISSION SOURCES - MAXIMUM ALLOWABLE EMISSION RATES
Permit Number 82739
This table lists the maximum allowable emission rates and all sources of air contaminants on the
applicants property covered by this permit. The emission rates shown are those derived from
information submitted as part of the application for permit and are the maximum rates allowed for
these facilities. Any proposed increase in emission rates may require an application for a
modification of the facilities covered by this permit.
AIR CONTAMINANTS DATA
Emission Source Air Contaminant Emission Rates
Point No. (1) Name (2) Name (3) lb/hr TPY(4)
LC-1 Lump Crusher (5) PM 0.02 0.03
PM 0.01 0.02
10
SC-2 Sifting Screen (5) PM 0.04 0.06
PM 0.01 0.02
10
LC-3 Conveyor Transfer (5) PM <0.01 <0.01
PM <0.01 <0.01
10
1 Cement Silo Baghouse PM 0.01 0.02
10
4, 5 Sand and Aggregate PM 0.39 0.58
Transfer To Hopper (5) PM 0.19 0.28
10
2, 6 Sand/Aggregate PM --- 0.02
Stockpile (5) PM 0.01
10


And here is the table corresponding to that example:

"Emission Source","Source Name","Air Contaminant Name","Emission Rate lbs/hr","Emission Rate tons/year"
LC-1,"Lump Crusher (5)","PM",0.02,0.03
LC-1,"Lump Crusher (5)","PM10",0.01,0.02
SC2,"Sifting Screen (5)","PM",0.04,0.06
SC2,"Sifting Screen (5)","PM10",0.01,0.02
LC-3,"Conveyor Transfer (5)","PM",<0.01,<0.01
LC-3,"Conveyor Transfer (5)","PM10",<0.01,<0.01
1,"Cement Silo Baghouse","PM",0.01,0.02 
4,5,"Sand and Aggregate Transfer To Hopper (5)","PM",0.39,0.58
4,5,"Sand and Aggregate Transfer To Hopper (5)","PM10",0.19,0.28


Now follows the full text:

{fulltext}

Please answer with the extracted table only.
"""


In [12]:
answer = query(prompt)

In [15]:
print(answer)

"Emission Source","Source Name","Air Contaminant Name","Emission Rate lbs/hr","Emission Rate tons/year"
LC-1,"Lump Crusher (5)","PM",0.02,0.03
LC-1,"Lump Crusher (5)","PM10",0.01,0.02
SC-2,"Sifting Screen (5)","PM",0.04,0.06
SC-2,"Sifting Screen (5)","PM10",0.01,0.02
LC-3,"Conveyor Transfer (5)","PM",<0.01,<0.01
LC-3,"Conveyor Transfer (5)","PM10",<0.01,<0.01
1,"Cement Silo Baghouse","PM",0.01,0.02 
4, 5,"Sand and Aggregate Transfer To Hopper (5)","PM",0.39,0.58
4, 5,"Sand and Aggregate Transfer To Hopper (5)","PM10",0.19,0.28
2, 6,"Sand/Aggregate Stockpile (5)","PM",---,0.02
2, 6,"Sand/Aggregate Stockpile (5)","PM10",---,0.01
3, 7,"Iron Ore Stockpile (5)","PM",---,1.07
3, 7,"Iron Ore Stockpile (5)","PM10",0.54,---
8,"Adhesive Applicator","VOC",5.88,8.73
8,"Adhesive Applicator","PM",5.30,0.16
MSS,"Cleaning Solvents","VOC",14.34,2.13
MSS,"Cleaning Solvents","Exempt Solvent",13.20,1.96
ALL,"SITE-WIDE","Individual HAP",<10.00,---
ALL,"SITE-WIDE","Combined HAPs",<25.00,---